<a href="https://colab.research.google.com/github/Trample86/TTS/blob/main/Ticketing_System_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
from datetime import datetime
import bcrypt
from tabulate import tabulate
global username, role

def connect_db():
  conn = sqlite3.connect('ticketing.db')
  cur = conn.cursor()
  cur.execute('''CREATE TABLE IF NOT EXISTS
  user(username TEXT PRIMARY KEY, password TEXT, role TEXT)''') # user table
  cur.execute('''CREATE TABLE IF NOT EXISTS
  staff(username TEXT PRIMARY KEY, indiv_ticket INTEGER, collab_ticket INTEGER)''') # staff table
  cur.execute('''CREATE TABLE IF NOT EXISTS
  tickets(ticketid TEXT PRIMARY KEY, location TEXT, equipment TEXT, no_of_staff INTEGER,
  status TEXT, time_reg TEXT, time_sol TEXT)''') # tickets table
  cur.execute('''CREATE TABLE IF NOT EXISTS
  chat(ticketid TEXT PRIMARY KEY, time TEXT, user TEXT, conversation TEXT)''') # store conversatioms
  conn.commit()
  return conn

def register_acc(conn):
  cur = conn.cursor()
  named = False
  while not named:
    username = input('Enter the username:')
    while username == '':
      print("Please enter a username.")
    cur.execute('SELECT * FROM user WHERE username =?',(username,))
    result = cur.fetchone()
    print(result)
    if result != ():
      print("This username is alreday taken. Please try again.") # prevents repeating names
    else:
      named = True
  password = input('Enter the password:').encode('utf-8')
  cryptedpw = bcrypt.hashpw(password, bcrypt.gensalt())
  cur.execute('''INSERT INTO user (username, password, role)
  VALUES (?,?,?)''', (username, cryptedpw, role)) #inputing the encrypted password
  conn.commit()
  print("Account added successfully!")

def login_acc(conn):
  global role
  cur = conn.cursor()
  found = False
  username = input('Enter your username:')
  cur.execute('SELECT username FROM user WHERE username =?',(username,))
  result = cur.fetchone()
  print(result)
  if result[0] == username:
    found = True
  else:
    print("This username is not found. Please try again.")
  correctpw = False
  cur.execute('SELECT password FROM user WHERE AND username =?',(username,))
  hashedpw = (cur.fetchone())[0]
  while not correctpw:
    attemptpw = input("Please enter your password:")
    if bcrypt.checkpw(attemptpw, hashedpw):
      correctpw == True
    else:
      print("Password does not match.")
  cur.execute('SELECT role FROM user WHERE username =?',(username,))
  role = cur.fetchone()
  role = role[2]
  return role

def add_tickets(conn): # adds the ticket
  cur = conn.cursor()
  location = (input("Please type the room number where the issus occurs: ")).strip().upper()
  equipment = (input("Please type the equipment that has an issue: ")).strip().title()
  now = datetime.now()
  time_reg = now.strftime("%Y-%m-%d %H:%M:%S")
  cur.execute('''SELECT COUNT(*) FROM tickets''')
  ticket_count = str((cur.fetchall()))[2]
  ticketid = str(int(ticket_count) + 1)
  status = 'REQUESTED'
  cur.execute('''INSERT INTO tickets (ticketid, location, equipment, time_reg, status)
  VALUES (?,?,?,?,?)''', (ticketid, location, equipment,time_reg,status))
  send_msg(conn,ticketid)
  conn.commit()

def display_ticket_list(conn,username): # display the tickets as a list
  cur = conn.cursor()
  display_row = []
  headers = ['No.','Location', 'equipment','Status']
  tickets_list = []
  if role == 'USER':
    cur.execute('''SELECT ticketid,location,equipment,status,time_reg
    FROM tickets WHERE username =?,''',(username,))
    user_tickets = cur.fetchall()
    for i in user_tickets: # preparng the data to present them as a table
      current_row = f"{i[0]},{i[1]},{i[2]},{i[3]},{i[4]}"
      display_row.append(current_row.split(','))
      tickets_list += i[0]
      tickets_list = ', '.join(tickets_list)
    print(tabulate(display_row, headers=headers, tablefmt="fancy_grid", numalign = "center", stralign = "center"))
  elif role == 'STAFF':
    cur.execute('''SELECT ticketid,location,equipment,status,time_reg
    FROM tickets WHERE status = 'PROGRESSING' OR status = 'REQUESTED'
    ORDER BY status''')
    user_tickets = cur.fetchall()
    for i in user_tickets:
      current_row = f"{i[0]},{i[1]},{i[2]},{i[3]},{i[4]}"
      display_row.append(current_row.split(','))
      tickets_list += i[0]
      tickets_list = ', '.join(tickets_list)
    print(tabulate(display_row, headers=headers, tablefmt="fancy_grid", numalign = "center", stralign = "center"))
  return tickets_list

def display_ticket(conn,tickets_list): # let user choose one particular ticket
  cur = conn.cursor()
  display_row = []
  headers = ['Time sent','User', 'Message']
  foundt = False
  ticket_choice = input('Please type the number of the ticket you are viewing: ').strip()

  while not foundt: # finding the input among the tickets
    try:
      if ticket_choice not in tickets_list:
        print(f"Please enter the number out of the followings:{tickets_list}")
        ticket_choice = input('Please type the number of the ticket you are viewing: ').strip()
      else:
        foundt = True
    except Exception as e:
      print(f"Please enter the number out of the followings:{tickets_list}")
      ticket_choice = input('Please type the number of the ticket you are viewing: ').strip()
  cur.execute(f'''SELECT time_sent, username, message
  FROM chat WHERE ticketid =?''',(ticket_choice,))
  msg_list = cur.fetchall()
  for i in msg_list: # preparng the data to present them as a table
    current_row = f"{i[0]},{i[1]},{i[2]}"
    display_row.append(current_row.split(','))
  print(tabulate(display_row, headers=headers, tablefmt="fancy_grid", numalign = "center", stralign = "center"))
  cur.execute(f'''SELECT status FROM tickets WHERE ticketid =?''',(ticket_choice,))
  status = (cur.fetchone())[0]

  if status == 'SOLVED' and role == 'USER': # deleting the ticket once both user and staff saves it
      print("The ticket will be deleted and exported as a text file.")
      output_ticket(conn,ticket_choice)
      cur.execute('''DELETE * FROM chat where ticketid = ?''',(ticket_choice,))


def send_msg(conn, ticketid): # subroutine for sending msg in a ticket
  cur = conn.cursor()
  msg = (input("Please type your message: ")).strip()
  now = datetime.now()
  time_sent = now.strftime("%Y-%m-%d %H:%M:%S")
  cur.execute(f'''INSERT INTO chat (ticketid, time, user, message)
  VALUES (?,?,?,?)''', (ticketid, time_sent,username,msg))
  conn.commit()

def ticket_status(conn, ticketid):
  cur = conn.cursor()
  cur.execute(f'''SELECT status FROM quests WHERE questid =?''',(ticketid,))
  status = (cur.fetchone())[0]

  changed = False
  while not changed:
    if status == 'REQUESTED': # confiriming the ticket for staffs
      cur.execute(f'''UPDATE tickets SET status = "PROGRESSING",no_of_staff = "1"
      WHERE ticketid =?''',(ticketid,))
      changed = True

    elif status == 'PROGRESSING':
      cur.execute(f'''SELECT username FROM user
      WHERE role = "STAFF"''')
      no_of_staff = (cur.fetchone())[0]

      cur.execute(f'''UPDATE tickets SET no_of_staff = "{no_of_staff}"
      WHERE ticketid =?''',(ticketid,))
      solved = input("Is the ticket solved? (Yes/no)").strip().upper()
      if solved == 'YES':
        cur.execute(f'''SELECT indiv_ticket,collab_ticket FROM staff
        WHERE username =?''',(username,))
        tickets_done = cur.fetchone()
        indiv_ticket_done = (tickets_done)[0]
        collab_ticket_done= (tickets_done)[1]
        if no_of_staff == 1: # counting staff involved
          indiv_ticket_done += 1
          cur.execute(f'''UPDATE staff SET indiv_ticket = "{indiv_ticket_done}",
          WHERE username =?''',(username,))
        else:
          collab_ticket_done += 1
          cur.execute(f'''UPDATE staff SET collab_ticket = "{collab_ticket_done}",
          WHERE username =?''',(username,))

        now = datetime.now()
        time_sol = now.strftime("%Y-%m-%d %H:%M:%S")
        cur.execute(f'''UPDATE tickets SET status = "SOLVED", time_sol = "{time_sol}"
        WHERE ticketid =?''',(ticketid,))
        changed = True
        output_ticket(conn,ticketid)
      elif solved == 'NO':
        changed = True
      else:
        print("Please enter a valid input.")

def output_ticket(conn,ticketid): # export ticket as a text file
  cur = conn.cursor()
  fname = f'ticket_{ticketid}.txt'
  with open(f"{fname}",'a') as file:
    cur.execute(f'''SELECT time, user, conversation
    FROM chat WHERE ticketid = "{ticketid}" ORDER BY time DESC''')
    msg_list = cur.fetchall()
    for i in msg_list:
      current_row = f"Time: {i[0]},Username: {i[1]}\n{i[2]}"
      file.write(f'{current_row}')
    file.write("End of ticket.")

def close_db(conn):
  conn.close()

def main():
  global username, role
  conn = connect_db()
  while True:
    print('''
    1. Login account
    2. Exit''')
    user_action = input("Enter action:")
    if user_action == '1':
      role = login_acc(conn)
    elif user_action == '2':
      close_db(conn)
      break
    else:
      print("Invalid option. Please try again.")
    if role == 'USER':
      user_main()
    elif role == 'STAFF':
      staff_main()

def user_main():
  conn = connect_db()
  while True:
    print('''
    1. Make new ticket
    2. View your tickets
    3. Send message to ongoing tickets
    4. Exit''')
    user_action = input("Enter action:")
    if user_action == '1':
      add_tickets(conn)
    elif user_action == '2':
      display_ticket_list(conn,username)
      tickets_list = display_ticket(conn,tickets_list)
    elif user_action == '3':
      tickets_list = display_ticket_list(conn,username)
      ticketid = print(f"Please choose a ticket:{tickets_list}")
      send_msg(conn, ticketid)
    elif user_action == '4':
      close_db(conn)
      break
    else:
      print("Invalid option. Please try again.")

def staff_main():
  conn = connect_db()
  while True:
    print('''
    1. View tickets list
    2. Send message to ongoing tickets
    3. Add account
    4. Exit''')
    user_action = input("Enter action:")
    if user_action == '1':
      tickets_list = display_ticket_list(conn,username)
      display_ticket(conn,tickets_list)
    elif user_action == '2':
      tickets_list = display_ticket_list(conn,username)
      ticketid = print(f"Please choose a ticket:{tickets_list}")
      send_msg(conn, ticketid)
      ticket_status(conn, ticketid)
    elif user_action == '3':
      register_acc(conn)
    elif user_action == '4':
      close_db(conn)
      break
    else:
      print("Invalid option. Please try again.")


MessageError: Error: credential propagation was unsuccessful